# Импорт библиотек и загрузка данных

### В этом блоке кода импортируются необходимые библиотеки для работы с данными и машинным обучением, а также происходит загрузка набора данных apple_quality.csv.

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

data = pd.read_csv('apple_quality.csv')

print(data.head())

print(data.info())

   A_id      Size    Weight  Sweetness  Crunchiness  Juiciness  Ripeness  \
0     0 -3.970049 -2.512336   5.346330    -1.012009   1.844900  0.329840   
1     1 -1.195217 -2.839257   3.664059     1.588232   0.853286  0.867530   
2     2 -0.292024 -1.351282  -1.738429    -0.342616   2.838636 -0.038033   
3     3 -0.657196 -2.271627   1.324874    -0.097875   3.637970 -3.413761   
4     4  1.364217 -1.296612  -0.384658    -0.553006   3.030874 -1.303849   

    Acidity Quality  
0 -0.491590    good  
1 -0.722809    good  
2  2.621636     bad  
3  0.790723    good  
4  0.501984    good  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   A_id         4000 non-null   int64  
 1   Size         4000 non-null   float64
 2   Weight       4000 non-null   float64
 3   Sweetness    4000 non-null   float64
 4   Crunchiness  4000 non-null   float64
 5   Juicin

# Предобработка данных и подготовка к обучению

### Здесь данные разделяются на признаки (X) и целевую переменную (y). Целевая переменная Quality является категориальной, поэтому она кодируется с использованием LabelEncoder, преобразуя значения "good" и "bad" в числовые метки. Далее данные разделены на обучающую, валидационную и тестовую выборки в пропорции 80%, 10% и 10% соответственно. Признаки стандартизируются с помощью StandardScaler, чтобы обеспечить одинаковый масштаб для всех признаков и улучшить производительность модели.

In [11]:
X = data.drop('Quality', axis=1)
y = data['Quality']

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Определение структуры нейронной сети

### В данном блоке создается последовательная модель нейронной сети с использованием библиотеки Keras. Модель состоит из трех полносвязных слоев (Dense): первый слой содержит 64 нейрона с функцией активации ReLU, соответствующий числу входных признаков. Второй слой имеет 32 нейрона с функцией активации ReLU. Выходной слой содержит один нейрон с функцией активации Sigmoid, что делает его подходящим для задачи бинарной классификации.

In [12]:

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  
    Dense(32, activation='relu'),  
    Dense(1, activation='sigmoid') 
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())

c:\Users\dimap\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,689 (10.50 KB)

 Trainable params: 2,689 (10.50 KB)

 Non-trainable params: 0 (0.00 B)

None


# Обучение нейронной сети

### В этом блоке происходит обучение нейронной сети на обучающих данных. Процесс обучения продолжается в течение 50 эпох с размером пакета (batch_size) 16. Во время обучения также используется валидационная выборка для оценки производительности модели на отдельных данных.

In [13]:
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_val, y_val), verbose=1)

Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.6814 - loss: 0.5926 - val_accuracy: 0.8575 - val_loss: 0.3769
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8479 - loss: 0.3454 - val_accuracy: 0.8850 - val_loss: 0.3127
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8691 - loss: 0.2979 - val_accuracy: 0.8850 - val_loss: 0.3028
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8716 - loss: 0.2930 - val_accuracy: 0.8750 - val_loss: 0.2925
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8823 - loss: 0.2723 - val_accuracy: 0.8950 - val_loss: 0.2850
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8857 - loss: 0.2645 - val_accuracy: 0.8850 - val_loss: 0.2857
Epoch 7/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8824 - loss: 0.2678 - val_accuracy: 0.8975 - val_loss: 0.2815
Epoch 8/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8930 - loss: 0.2610 - val_accuracy: 0.

# Оценка модели и анализ точности

### После завершения обучения модели на тестовой выборке оценивается точность (Accuracy) модели. Также выводится отчет классификации (classification_report), включающий precision, recall, F1-score и поддержку для каждого класса, а также матрица ошибок (confusion_matrix), которая показывает, какие классы были правильно или неправильно предсказаны моделью.

In [14]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {accuracy:.4f}')

y_pred = model.predict(X_test)
y_pred_classes = np.round(y_pred) 

print("Classification Report:")
print(classification_report(y_test, y_pred_classes))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))

Test Accuracy: 0.9075
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.93      0.91       197
           1       0.93      0.88      0.91       203

    accuracy                           0.91       400
   macro avg       0.91      0.91      0.91       400
weighted avg       0.91      0.91      0.91       400

Confusion Matrix:
[[184  13]
 [ 24 179]]


# Сравнение с алгоритмами машинного обучения и выводы

### В этом последнем блоке происходит сравнение точности нейронной сети с другими алгоритмами машинного обучения, такими как логистическая регрессия и случайный лес. Каждый алгоритм оценивается на тестовой выборке, и их точности выводятся на экран. В конце выводится собственное заключение на основе полученных результатов, подчеркивая преимущества и ограничения каждой модели в данной задаче.

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)
lr_accuracy = lr_model.score(X_test, y_test)
print(f'Logistic Regression Accuracy: {lr_accuracy:.4f}')

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_accuracy = rf_model.score(X_test, y_test)
print(f'Random Forest Accuracy: {rf_accuracy:.4f}')

print("Сравнение с другими алгоритмами машинного обучения:")
print(f"Нейронная сеть Accuracy: {accuracy:.4f}")
print(f"Логистическая регрессия Accuracy: {lr_accuracy:.4f}")
print(f"Случайный лес Accuracy: {rf_accuracy:.4f}")

if accuracy > lr_accuracy and accuracy > rf_accuracy:
    print("Нейронная сеть показала лучший результат точности.")
elif rf_accuracy > lr_accuracy:
    print("Случайный лес показал лучший результат точности.")
else:
    print("Логистическая регрессия показала лучший результат точности.")

Logistic Regression Accuracy: 0.7750
Random Forest Accuracy: 0.8900
Сравнение с другими алгоритмами машинного обучения:
Нейронная сеть Accuracy: 0.9075
Логистическая регрессия Accuracy: 0.7750
Случайный лес Accuracy: 0.8900
Нейронная сеть показала лучший результат точности.
